In [ ]:
# servidor_grupo.ipynb

%run FunçõesAuxiliares.ipynb 

import socket
import threading
import ssl

# Entrada dinâmica dos membros do grupo
entrada = input("Digite os nomes dos membros do grupo separados por vírgula (ex: Alice,Bob,Marianna): ")
membros_grupo = [nome.strip() for nome in entrada.split(",") if nome.strip()]

print(f"🧑‍🤝‍🧑 Grupo configurado com membros: {membros_grupo}")

def distribuir_chaves_clientes(clientes):
    dic = chaves_clientes_grupo(clientes)
    for cliente, gsk_encriptada in dic.items():
        with open(f"gsk_encriptada_para_{cliente}.pem", "wb") as f:
            f.write(gsk_encriptada)

distribuir_chaves_clientes(membros_grupo)

NOME_SERVIDOR = "Marianna"
PATH_CERT = "certificados/Marianna_cert.pem"
PATH_KEY = "certificados/Marianna_chave.pem"
PATH_CA = "AutoridadeCertificadora/certificado_raiz.pem"
PATH_GSK = f"gsk_encriptada_para_{NOME_SERVIDOR}.pem"
PATH_GSK_ORIGINAL = "gsk_original.pem"

def carregar_gsk(path_gsk):
    with open(path_gsk, "rb") as f:
        return f.read()

clientes_conectados = []  # lista de (socket, endereço)
gsk_desencriptada = carregar_gsk(PATH_GSK_ORIGINAL)

def broadcast(mensagem_cifrada, remetente):
    for cliente, _ in clientes_conectados:
        if cliente != remetente:
            try:
                cliente.send(mensagem_cifrada)
            except:
                cliente.close()
                clientes_conectados.remove((cliente, _))

def lidar_com_cliente(cliente_socket, endereco):
    print(f"📥 Novo cliente conectado: {endereco}")
    while True:
        try:
            mensagem_cifrada = cliente_socket.recv(4096)
            mensagem = desencriptar_mensagem_grupo(mensagem_cifrada, gsk_desencriptada)
            print(f"🗣️ {endereco}: {mensagem}")
            broadcast(mensagem_cifrada, cliente_socket)
        except:
            print(f"❌ Cliente {endereco} desconectado.")
            cliente_socket.close()
            clientes_conectados.remove((cliente_socket, endereco))
            break

def iniciar_servidor():

    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile=PATH_CERT, keyfile=PATH_KEY)
    context.load_verify_locations(cafile=PATH_CA)
    context.verify_mode = ssl.CERT_REQUIRED
    
    servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    servidor.bind(("0.0.0.0", 4444))
    servidor.listen(3)
    print("🚀 Servidor de grupo aguardando conexões...")

    while True:
        cliente_socket, endereco = servidor.accept()
        try:
            conexao_ssl = context.wrap_socket(cliente_socket, server_side=True)
            clientes_conectados.append((conexao_ssl, endereco))
            thread = threading.Thread(target=lidar_com_cliente, args=(conexao_ssl, endereco))
            thread.start()
        except ssl.SSLError as e:
            print(f"❌ Erro SSL ao aceitar conexão de {endereco}: {e}")
            cliente_socket.close()

if __name__ == "__main__":
    iniciar_servidor()


Digite os nomes dos membros do grupo separados por vírgula (ex: Alice,Bob,Marianna):  Alice,Bob,Lourenco


🧑‍🤝‍🧑 Grupo configurado com membros: ['Alice', 'Bob', 'Lourenco']
🚀 Servidor de grupo aguardando conexões...
📥 Novo cliente conectado: ('127.0.0.1', 45265)
🗣️ ('127.0.0.1', 45265): Lourenco: oi
📥 Novo cliente conectado: ('127.0.0.1', 45277)
🗣️ ('127.0.0.1', 45277): Alice: oi
🗣️ ('127.0.0.1', 45265): Lourenco: :DD
🗣️ ('127.0.0.1', 45265): Lourenco: ainda bem
🗣️ ('127.0.0.1', 45277): Alice: que ja funfa
🗣️ ('127.0.0.1', 45265): Lourenco: juro
📥 Novo cliente conectado: ('127.0.0.1', 45366)
🗣️ ('127.0.0.1', 45366): Bob: mekie malta
🗣️ ('127.0.0.1', 45265): Lourenco: GANDA BOB PÁ
🗣️ ('127.0.0.1', 45277): Alice: como tamos?
🗣️ ('127.0.0.1', 45265): Lourenco: tamos bem
🗣️ ('127.0.0.1', 45366): Bob: vou bazar ficaram
❌ Cliente ('127.0.0.1', 45366) desconectado.
🗣️ ('127.0.0.1', 45277): Alice: eu tmb
❌ Cliente ('127.0.0.1', 45265) desconectado.
❌ Cliente ('127.0.0.1', 45277) desconectado.
